# VGG для MNIST
Используем обученную нейросеть VGG без последнего слоя для классификации цифр.

Изменим размеры изображений до 32x32 (5 слоев подвыборки) и приведем их к полноцветным. Обучим последние слои нейросети на этом наборе и проверим точность.

![VGG](vgg16-neural-network-1.jpg)

### Подключение библиотек

In [1]:
import numpy as np
import pandas as pd
import keras as K
from sklearn.metrics import classification_report
import cv2

Using TensorFlow backend.


### Загрузка набора MNIST

In [2]:
(X_train, y_train), (X_test, y_test) = K.datasets.mnist.load_data()

### Конвертация изображений
Серые 28x28 -> полноцветные 32x32

In [5]:
def convert_data (x):
    data_rgb = []
    for i in x:
        img = cv2.resize(i, (32, 32), interpolation=cv2.INTER_AREA)
        rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
        data_rgb.append(rgb)
    data_rgb = np.stack([data_rgb], axis=4)
    return np.squeeze(data_rgb, axis=4)

In [6]:
data_rgb = convert_data(X_train)
print (data_rgb.shape)

(60000, 32, 32, 3)


### Загрузка обученной модели
Без последнего слоя - его и будем обучать

In [7]:
base_model = K.applications.vgg16.VGG16(weights="imagenet", include_top=False)

### Фиксация всех текущих весов модели
И добавление последнего слоя для классификации

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [13]:
x = base_model.output
x = K.layers.GlobalAveragePooling2D()(x)
x = K.layers.Dense(4096, activation="relu")(x)
x = K.layers.Dense(10, activation="softmax")(x)
model = K.models.Model(inputs=base_model.input, outputs=x)
model.compile(optimizer="SGD", loss=K.losses.categorical_crossentropy)

### Обучение нейросети

In [14]:
model.fit(data_rgb, K.utils.to_categorical(y_train, 10), batch_size=5000, epochs=100)

Epoch 1/100
60000/60000 [==============================] - 223s 4ms/step - loss: 23.3241
Epoch 2/100
60000/60000 [==============================] - 211s 4ms/step - loss: 0.7039
Epoch 3/100
60000/60000 [==============================] - 210s 4ms/step - loss: 0.5751
Epoch 4/100
60000/60000 [==============================] - 209s 3ms/step - loss: 0.4794
Epoch 5/100
60000/60000 [==============================] - 209s 3ms/step - loss: 0.4438
Epoch 6/100
60000/60000 [==============================] - 210s 4ms/step - loss: 0.4480
Epoch 7/100
60000/60000 [==============================] - 209s 3ms/step - loss: 0.3910
Epoch 8/100
60000/60000 [==============================] - 209s 3ms/step - loss: 0.3684
Epoch 9/100
60000/60000 [==============================] - 210s 4ms/step - loss: 0.3244
Epoch 10/100
60000/60000 [==============================] - 209s 3ms/step - loss: 0.3895
Epoch 11/100
60000/60000 [==============================] - 209s 3ms/step - loss: 0.3108
Epoch 12/100
60000/60000 [===

60000/60000 [==============================] - 209s 3ms/step - loss: 0.1197
Epoch 94/100
60000/60000 [==============================] - 208s 3ms/step - loss: 0.1175
Epoch 95/100
60000/60000 [==============================] - 209s 3ms/step - loss: 0.1164
Epoch 96/100
60000/60000 [==============================] - 209s 3ms/step - loss: 0.1160
Epoch 97/100
60000/60000 [==============================] - 208s 3ms/step - loss: 0.1170
Epoch 98/100
60000/60000 [==============================] - 210s 3ms/step - loss: 0.1208
Epoch 99/100
60000/60000 [==============================] - 209s 3ms/step - loss: 0.1164
Epoch 100/100
60000/60000 [==============================] - 209s 3ms/step - loss: 0.1144


### Предсказание значений и оценка

In [15]:
result = np.argmax(model.predict(convert_data(X_test)), axis=1)
print (classification_report(result, y_test, digits=4))

              precision    recall  f1-score   support

           0     0.9786    0.9897    0.9841       969
           1     0.9912    0.9817    0.9864      1146
           2     0.9312    0.9459    0.9385      1016
           3     0.9525    0.9295    0.9408      1035
           4     0.9613    0.9555    0.9584       988
           5     0.9137    0.9076    0.9106       898
           6     0.9635    0.9655    0.9645       956
           7     0.9572    0.9638    0.9605      1021
           8     0.9559    0.9608    0.9583       969
           9     0.9524    0.9591    0.9557      1002

    accuracy                         0.9565     10000
   macro avg     0.9557    0.9559    0.9558     10000
weighted avg     0.9565    0.9565    0.9565     10000

